In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
#Reading the csv file into a dataframe
data = pd.read_csv('../input/mines-vs-rocks/sonar.all-data.csv', header=None)

In [ ]:
# View the top five rows of the data
data.head()

In [ ]:
X = data.drop(60, axis=1)

In [ ]:
print(X.shape)
X.head()

In [ ]:
Y = data[60]
print(Y.head())
Y.replace({'R':0, 'M':1}, inplace=True)

In [ ]:
Y

In [ ]:
from sklearn.model_selection import train_test_split
X_trn, X_tst, y_trn, y_tst = train_test_split(X, Y, test_size=0.2, random_state=0)

In [ ]:
def sigmoid(x):
    '''
    Returns the sigmoid of x
    '''
    return 1/(1+np.exp(-x))

In [ ]:
def init_param(X):
    '''
    Args: X -> input matrix
    Returns: a tuple of initialised parameters , w vector and b for the bias
    '''
    np.random.seed(0)
    w = np.random.rand(X.shape[1])
    b = 0
    param = {'w': w,
             'b': b
            }
    return param


In [ ]:
def Cost_func(X, Y, param):
    '''
    Computes the cost and gradient value for a given X, Y, param
    Args:
        X => input feautre matrix 
        Y => output array
        param => parameters
    Returns: 
        J => Cost 
        grad => gradient values for w and b
    '''
    m = len(X)
    prob = np.dot(param['w'], X.T) + param['b']
    pred = sigmoid(prob)
    
    J = - np.sum((Y* np.log(pred)) + ((1-Y)*(np.log(1-pred))))/m
    
    dw = (1/m)* np.dot((pred - Y), X)
    db = (1/m)* np.sum(pred -Y)
    
    grad= {'dw': dw,
           'db': db}
    
    return J, grad

In [ ]:
def optimize(X, Y, param, num_iter, learning_rate):
    '''
    Optimize the paramenter using gradient descent.
    Args:
        X => input feature matrix
        Y => output array
        param => parameters which are updated
        num_iter => total number of itertions
        learning_rate => learning rate to specify the step size during gradient descent
    Returns:
        param => updated parameters' array
        cost => final cost        
    '''
    for i in range(num_iter):
        cost, grad = Cost_func(X, Y, param)
        param['w'] = param['w']-(learning_rate*grad['dw'])
        param['b'] = param['b']-(learning_rate*grad['db'])
        
        
    return param, cost

In [ ]:
def predict(X, param):
    '''
    predicts the target value, Y_pred
    Args: X => input feature matrix,
          param => parameter vector (w, b)
    Returns: Array of predictions
    '''
    Y_pred=[]
    prob= np.dot(param['w'], X.T) + param['b']
    pred = sigmoid(prob)
    for i in range(len(X)):
        if pred[i] > 0.5:
            Y_pred.append(1)
        else:
            Y_pred.append(0)
            
    return np.array(Y_pred).T
        

In [ ]:
def accuracy(Y, Y_pred):
    '''
    Returns the accuracy for the predictions
    '''
    return np.sum(Y == Y_pred) / len(Y)

In [ ]:
def main(X, Y, learning_rate=0.01, num_iter=5000):
    '''
    Combining all the possible 
    '''
    param = init_param(X)
    cost, grad= Cost_func(X, Y, param)
    param, cost_f = optimize(X, Y, param, num_iter, learning_rate)
    
    y_pred = predict(X, param)
    acc = accuracy(Y, y_pred)
    
    print('Final_cost \t', cost_f)
    print('Accuracy_train \t', acc)
    return param, y_pred

In [ ]:
param_f = main(X_trn, y_trn)[0]

In [ ]:
pred_y = predict(X_tst, param_f)

In [ ]:
accuracy(y_tst, pred_y)